In [12]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
import coralme

In [13]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# !bash report.sh
organisms = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
done = set(i.split(".csv")[0] for i in os.listdir("./cases/fluxes/nonIBD/") if "csv" in i)

In [15]:
len(organisms)

495

### Load

In [16]:
def load_data(path):
    try:
        df = pandas.read_csv(path,index_col=0)[["fluxes"]]
    except:
        df = pandas.DataFrame(columns=["fluxes"])
    df.columns = [org]
    return df

In [17]:
from tqdm import tqdm
conditions = ["nonIBD", "IBD"]

In [18]:
overflow = {i:pandas.DataFrame() for i in conditions}
growth = {i:{} for i in conditions}

In [19]:
for org in tqdm(organisms):
    for c in conditions:
        tmp = load_data("./cases/overflow/{}/{}.csv".format(c,org))
        overflow[c] = pandas.concat([overflow[c],tmp], axis=1).fillna(0)

100%|██████████| 495/495 [00:01<00:00, 284.81it/s]


In [20]:
for org in tqdm(organisms):
    for c in conditions:
        tmp = load_data("./cases/fluxes/{}/{}.csv".format(c,org))
        if tmp.empty:
            val = None
        else:
            val = tmp[org]["biomass_dilution"]
        growth[c][org] = val
growth = pandas.DataFrame.from_dict(growth)

100%|██████████| 495/495 [00:30<00:00, 16.09it/s]


In [21]:
growth.dropna()

,nonIBD,IBD
Dialister_succinatiphilus_YIT_11850,0.000984,0.001566
Lactobacillus_coleohominis_101_4_CHN,0.017685,0.028927
Haemophilus_parainfluenzae_T3T1,0.148266,0.316874
Bacteroides_thetaiotaomicron_VPI_5482,0.132143,0.205156
Enterococcus_faecalis_TX1322,0.001426,0.000438
...,...,...
Clostridium_sticklandii_DSM_519,0.002309,0.000878
Staphylococcus_pettenkoferi_VCU012,0.002585,0.001740
Bacteroides_sp_2_1_7,0.093961,0.073051
Actinobacillus_pleuropneumoniae_L20,0.001815,0.001720


### Save

In [22]:
growth.sort_index().to_csv("datasets/3.5.IBD_growth_rates.csv")

In [23]:
for c,df in overflow.items():
    df.to_csv("datasets/3.5.{}_overflow.csv".format(c))

In [25]:
growth = pandas.read_csv("datasets/3.5.IBD_growth_rates.csv",index_col=0)
growth

,nonIBD,IBD
Abiotrophia_defectiva_ATCC_49176,0.001993,0.001793
Acidaminococcus_fermentans_DSM_20731,0.001525,0.001096
Acidaminococcus_intestini_RyC_MR95,0.001026,0.002054
Acidaminococcus_sp_D21,0.002297,0.043386
Acinetobacter_calcoaceticus_PHEA_2,0.001129,0.001091
...,...,...
Yersinia_enterocolitica_subsp_enterocolitica_8081,0.002792,0.366087
Yersinia_frederiksenii_ATCC_33641,0.004153,0.003894
Yersinia_kristensenii_ATCC_33638,0.000870,0.032794
Yersinia_pseudotuberculosis_YPIII,0.007851,0.001181


In [7]:
growth = pandas.read_csv("datasets/3.5.IBD_growth_rates.csv",index_col=0)

,nonIBD,IBD
Abiotrophia_defectiva_ATCC_49176,0.001993,NaN
Acidaminococcus_fermentans_DSM_20731,0.001525,0.001937
Acidaminococcus_intestini_RyC_MR95,0.001026,0.002054
Acidaminococcus_sp_D21,0.002297,0.055847
Acinetobacter_calcoaceticus_PHEA_2,0.001129,0.000918
...,...,...
Yersinia_enterocolitica_subsp_enterocolitica_8081,0.002792,0.001391
Yersinia_frederiksenii_ATCC_33641,0.004153,0.003634
Yersinia_kristensenii_ATCC_33638,0.000870,0.001085
Yersinia_pseudotuberculosis_YPIII,0.007851,0.001409
